# 1.1 chABSA-datasetから訓練、テストデータを作成

- 本ファイルでは、chABSA-datasetのデータを使用して、感情分析（0：ネガティブ、1：ポジティブ）を2値クラス分類するためのデータファイル（tsv)を作成します。

- 下記サイトからchABSA-dataset.zipをダウンロードして解凍します。

     https://github.com/chakki-works/chABSA-dataset

     データファイルは230個、文章データは2813個あります。



In [1]:
#pathの追加（必要に応じて）
import sys
sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python36.zip')
sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6')
sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6/lib-dynload')
sys.path.append('/home/siny/.local/lib/python3.6/site-packages')
sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6/site-packages')

In [2]:
import pandas as pd
from pathlib import Path
import json

In [3]:
data_dir = "data/chABSA/"
data_path = Path(data_dir)
path_list = [p for p in data_path.iterdir()]
print("ファイル数＝",len(path_list))
print("ファイルチェック",path_list[:5])

ファイル数＝ 230
ファイルチェック [PosixPath('data/chABSA/e00008_ann.json'), PosixPath('data/chABSA/e00017_ann.json'), PosixPath('data/chABSA/e00024_ann.json'), PosixPath('data/chABSA/e00026_ann.json'), PosixPath('data/chABSA/e00030_ann.json')]


In [4]:
with open(path_list[0], "br") as f:
    j =  json.load(f)
sentences = j["sentences"]  # sentencesにはデータ数分のデータが格納されている。
print("1つ目のファイル内のデータ数チェック：",len(sentences))

1つ目のファイル内のデータ数チェック： 36


In [5]:
sentences[0]

{'sentence_id': 0,
 'sentence': '当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・所得環境は改善し、景気も緩やかな回復基調のうちに推移いたしましたが、中国をはじめとするアジア新興国経済の減速懸念や、英国の欧州連合（ＥＵ）離脱決定、米国新政権への移行など、引き続き先行きは不透明な状況となっております',
 'opinions': [{'target': 'わが国経済',
   'category': 'NULL#general',
   'polarity': 'neutral',
   'from': 11,
   'to': 16},
  {'target': '企業業績',
   'category': 'NULL#general',
   'polarity': 'positive',
   'from': 38,
   'to': 42},
  {'target': '雇用',
   'category': 'NULL#general',
   'polarity': 'positive',
   'from': 43,
   'to': 45},
  {'target': '所得環境',
   'category': 'NULL#general',
   'polarity': 'positive',
   'from': 46,
   'to': 50},
  {'target': '景気',
   'category': 'NULL#general',
   'polarity': 'positive',
   'from': 55,
   'to': 57}]}

In [6]:
sentences[1]

{'sentence_id': 1,
 'sentence': '当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節約志向により、個人消費は本格的な改善には至らず、また少子高齢化、人口減少による社会構造の変化、雇用改善に伴う労働コストの上昇、企業間競争の激化など、引き続き厳しい状況となりました',
 'opinions': [{'target': '環境',
   'category': 'NULL#general',
   'polarity': 'negative',
   'from': 11,
   'to': 13},
  {'target': '実質賃金',
   'category': 'NULL#general',
   'polarity': 'negative',
   'from': 15,
   'to': 19},
  {'target': '個人消費',
   'category': 'NULL#general',
   'polarity': 'negative',
   'from': 53,
   'to': 57},
  {'target': '労働コスト',
   'category': 'NULL#cost',
   'polarity': 'negative',
   'from': 100,
   'to': 105},
  {'target': '企業間競争',
   'category': 'NULL#general',
   'polarity': 'negative',
   'from': 109,
   'to': 114}]}

## 感情スコアの算出関数
- porarityがpositiveの場合スコア(rating)を+1、negativeだったら-1する。
- ratingには[スコア,文章]の形で格納される。
- 以下のようにsentenceに改行コード「\n」が含まれるデータがあると文章の抽出がおかしくなるのでsentenceに改行コードが含まれていた場合は除外することとする。

"sentence": "〈ホテル関連事業〉\n　ホテル関連事業部門では、国内外からの観光客の動きも殆ど落ち込むこともなく、宿泊部門は各ホテルとも堅調に推移しました",


In [7]:
def create_rating(sentences):
    rating = []
    for obj in sentences:
        s = obj["sentence"]  #文章の取得
        s = s.replace("\n","")
        op = obj["opinions"]  #options部分を取得
        porarity = 0
        for o in op:
            p = o["polarity"]
            if p == "positive":
                porarity += 1
            elif p == "negative":
                porarity -= 1
        if porarity !=0 :
            rating.append((porarity, s))
    return rating

In [8]:
#1つ目のファイルだけチェック
create_rating(sentences)

[(4,
  '当連結会計年度におけるわが国経済は、政府の経済政策や日銀の金融緩和策により、企業業績、雇用・所得環境は改善し、景気も緩やかな回復基調のうちに推移いたしましたが、中国をはじめとするアジア新興国経済の減速懸念や、英国の欧州連合（ＥＵ）離脱決定、米国新政権への移行など、引き続き先行きは不透明な状況となっております'),
 (-5,
  '当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節約志向により、個人消費は本格的な改善には至らず、また少子高齢化、人口減少による社会構造の変化、雇用改善に伴う労働コストの上昇、企業間競争の激化など、引き続き厳しい状況となりました'),
 (2,
  '以上の結果、当連結会計年度の当社グループの業績は、売上高631億19百万円（前期比3.5％増）、営業利益34億30百万円（前期比0.1％増）、経常利益43億79百万円（前期比7.5％増）、親会社株主に帰属する当期純利益は28億28百万円（前期比8.9％増）となりました'),
 (1,
  'なお、当連結会計年度の生産量は、ブナピーを含めブナシメジ42,602ｔ（同5.5％増）、エリンギ19,250ｔ（同0.2％減）、マイタケ14,281ｔ（同4.3％増）となりました'),
 (1,
  '平成27年４月の火災により生産を休止していた苫小牧第一きのこセンターが、工場を再建し、平成28年４月27日よりブナシメジの収穫・出荷を再開したことや、カットブナシメジ専用工場として新設した富山きのこセンターが平成28年９月上旬から収穫・出荷を開始したことにより、ブナシメジの生産量は増加いたしました'),
 (1,
  'また、改修のため一時生産を休止しておりました広川きのこセンターにおきまして、平成28年９月上旬より霜降りひらたけの収穫・出荷を開始したことなどにより、きのこ全体の生産量は増加いたしました'),
 (-2, '春から夏にかけましては個人消費の低迷などにより、きのこの価格は厳しい状況で推移いたしました'),
 (2, '反面、秋から冬にかけましては天候不順などによる野菜価格の高騰により、きのこの価格は堅調に推移いたしました'),
 (1, '以上の結果、国内きのこ事業全体の売上高は422億96百万円（同3.1％増）となりました'),
 (

In [9]:
#全ファイルからratingと文章を抽出
rating = []
for p in path_list:
    with open(p, "br") as f:
        j =  json.load(f)
    s = j["sentences"]
    rating += create_rating(s)


In [10]:
print(len(rating))

2813


In [11]:
#ネガポジの分類
neg =[]
pos = []
for t in rating:
    if t[0] > 0:
        pos.append(t[1])
    else:
        neg.append(t[1])

In [12]:
print("ネガティブデータ数=",len(neg))
print("ポジティブデータ数=",len(pos))

ネガティブデータ数= 1143
ポジティブデータ数= 1670


## 訓練データとテストデータを生成
- 今回は訓練データを70%、テストデータを30%の割合で分割する。
- train.tsv(訓練データ7割）
- test.tsv(テストデータ3割）

In [13]:
#ネガポジデータの生成(訓練データ)
threshold = 0.7
neg_num = len(neg)
pos_num = len(pos)

with open("data/chABSA/train.tsv", "a") as f:
    index = 0
    for text in neg:
        if index < neg_num * threshold:
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            break
    index = 0
    for text in pos:
        if index < pos_num * threshold:
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            break


In [14]:
#ネガポジデータの生成(テストデータ)
threshold = 0.7
neg_num = len(neg)
pos_num = len(pos)

with open("data/chABSA/test.tsv", "a") as f:
    index = 0
    for text in neg:
        if index >= neg_num * threshold:
            text = text+'\t'+'0'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            index += 1
    index = 0
    for text in pos:
        if index >= pos_num * threshold:
            text = text+'\t'+'1'+'\t'+'\n'
            f.write(text)
            index += 1
        else:
            index += 1